In [12]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import fetch_openml

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [13]:

# loading data

def load_data():
    diabetes_df = pd.read_csv('dataset/diabetes_prediction_dataset.csv')
    y = diabetes_df['diabetes'].values
    X = diabetes_df.drop('diabetes', axis=1)
    return X, y

# one hot vector
def one_hot_encode(X, columns):
    for col in columns:
        one_hot = pd.get_dummies(X[col], prefix=col)
        X = pd.concat([X, one_hot], join='inner', axis='columns').drop(col, axis=1)
    return X

# data cleaning

def remove_outliers(X):
    outlier_detection = IsolationForest()
    outliers = outlier_detection.fit_predict(X)

    X = X[outliers == 1]
    return X

def standardize_data(X, columns):
    scaler = MinMaxScaler()
    X[columns] = scaler.fit_transform(X[columns])
    return X

# data preparation

def prepare_data(X, columns_to_encode, columns_to_scale, columns_no_scale):
    X_encoded = one_hot_encode(X, columns_to_encode)

    X_scaled = standardize_data(X_encoded, columns_to_scale)


    X_final = pd.concat([X_scaled, X_encoded[columns_no_scale]], join='inner', axis='columns')
    return X_final

# split train and dev sets

def split_train_dev_test(X, y):
    total_examples = len(X)

    train_size = int(total_examples * 0.8)
    dev_size = int(total_examples * 0.1)
    Xtrain, Xdev, Xtest = X[:train_size], X[train_size:train_size+dev_size], X[train_size+dev_size:]
    ytrain, ydev, ytest = y[:train_size], y[train_size:train_size+dev_size], y[train_size+dev_size:]

    return Xtrain, Xdev, Xtest, ytrain, ydev, ytest


In [14]:
X, y = load_data()

columns_to_encode = ['gender', 'smoking_history']
columns_to_scale = ['age', 'bmi', 'HbA1c_level', 'blood_glucose_level']
columns_no_scale = ['hypertension', 'heart_disease']

X_prep = prepare_data(X, columns_to_encode, columns_to_scale, columns_no_scale)


Xtrain, Xdev, Xtest, ytrain, ydev, ytest = split_train_dev_test(X_prep, y)


print("Xtrain.shape, ytrain.shape: ", Xtrain.shape, ytrain.shape)
print("Xdev.shape, ydev.shape: ", Xdev.shape, ydev.shape)
print("Xtest.shape, ytest.shape: ", Xtest.shape, ytest.shape)

Xtrain = Xtrain.values.astype(np.float32)
Xdev = Xdev.values.astype(np.float32)
Xtest = Xtest.values.astype(np.float32)

Xtrain.shape, ytrain.shape:  (80000, 17) (80000,)
Xdev.shape, ydev.shape:  (10000, 17) (10000,)
Xtest.shape, ytest.shape:  (10000, 17) (10000,)


In [4]:
def grid_search_gradient_boosting(X_train, y_train):
    param_grid = {
        'n_estimators': [200, 300,500],
        'learning_rate': [0.01, 0.1, 0.2,0.6],
        'max_depth': [3, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
    }
    grid = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid, cv=5, refit= True, scoring='recall', n_jobs=-1, verbose=3)
    grid.fit(X_train, y_train)
    print("Grid searching is done!")
    print("The best score: ", grid.best_score_)
    print("The best hyperparameters:")
    print(grid.best_params_)
    return grid

best_model = grid_search_gradient_boosting(Xtrain, ytrain)
final_predictions = best_model.predict(Xtest)
accuracy = accuracy_score(ytest, final_predictions)
precision = precision_score(ytest, final_predictions, average='macro')
recall = recall_score(ytest, final_predictions, average='macro')
f1 = f1_score(ytest, final_predictions, average='macro')

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

print(best_model)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Grid searching is done!
The best score:  0.7114600453310131
The best hyperparameters:
{'learning_rate': 0.6, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Accuracy:  0.9658
Precision:  0.9120585554796081
Recall:  0.8635401506806358
F1 Score:  0.885910919245747
GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2, 0.6],
                         'max_depth': [3, 5, 10], 'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [200, 300, 500]},
             scoring='recall', verbose=3)
[CV 3/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=0.648 total time= 1.1min
[CV 1/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=2, min_samples_split=5, n_estimato

/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[CV 2/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=300;, score=0.678 total time= 1.1min
[CV 5/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=2, min_samples_split=5, n_estimators=200;, score=0.678 total time=  45.3s
[CV 4/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=2, min_samples_split=10, n_estimators=500;, score=0.669 total time= 1.9min
[CV 5/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=1, min_samples_split=2, n_estimators=500;, score=0.682 total time= 2.8min
[CV 5/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=2, min_samples_split=10, n_estimators=200;, score=0.678 total time= 1.1min
[CV 4/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=4, min_samples_split=2, n_estimators=500;, score=0.674 total time= 2.6min
[CV 2/5] END learning_rate=0.01, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=0.698 total time= 2.5min
[CV 2/5] END learning_rate=0.01, max_d

/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[CV 2/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=5, n_estimators=500;, score=0.678 total time= 1.7min
[CV 5/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=2, min_samples_split=10, n_estimators=200;, score=0.678 total time=  40.9s
[CV 3/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=4, min_samples_split=5, n_estimators=200;, score=0.648 total time=  40.4s
[CV 1/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=4, min_samples_split=10, n_estimators=300;, score=0.672 total time= 1.0min
[CV 2/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=500;, score=0.682 total time= 2.6min
[CV 5/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=2, min_samples_split=10, n_estimators=300;, score=0.678 total time= 1.6min
[CV 5/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=4, min_samples_split=10, n_estimators=200;, score=0.678 total time= 1.1min
[CV 5/5] END learning_rate=0.01, max_

...

/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[CV 3/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=10, n_estimators=200;, score=0.648 total time=  41.3s
[CV 3/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=10, n_estimators=500;, score=0.648 total time= 1.7min
[CV 5/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=4, min_samples_split=2, n_estimators=500;, score=0.678 total time= 1.7min
[CV 4/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=300;, score=0.669 total time= 1.6min
[CV 4/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=2, min_samples_split=5, n_estimators=200;, score=0.669 total time= 1.1min
[CV 1/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=2, min_samples_split=10, n_estimators=500;, score=0.677 total time= 2.7min
[CV 2/5] END learning_rate=0.01, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=500;, score=0.698 total time= 4.4min
[CV 5/5] END learning_rate=0.01, max_

/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[CV 4/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=5, n_estimators=500;, score=0.669 total time= 1.9min
[CV 4/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=4, min_samples_split=2, n_estimators=200;, score=0.669 total time=  44.5s
[CV 4/5] END learning_rate=0.01, max_depth=3, min_samples_leaf=4, min_samples_split=5, n_estimators=300;, score=0.669 total time= 1.1min
[CV 2/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=1, min_samples_split=5, n_estimators=200;, score=0.678 total time= 1.1min
[CV 5/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=1, min_samples_split=10, n_estimators=500;, score=0.683 total time= 2.9min
[CV 3/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=4, min_samples_split=5, n_estimators=200;, score=0.648 total time= 1.2min
[CV 1/5] END learning_rate=0.01, max_depth=5, min_samples_leaf=4, min_samples_split=10, n_estimators=500;, score=0.677 total time= 2.6min
[CV 1/5] END learning_rate=0.01, max_de

/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
def grid_search_adaboost(X_train, y_train):
    param_grid = {
        'base_estimator': [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2)],
        'n_estimators': [50, 100, 200, 500,800],
        'learning_rate': [0.01, 0.05, 0.1, 0.5, 0.8]
    }
    grid = GridSearchCV(
        AdaBoostClassifier(random_state=42), 
        param_grid, 
        cv=5, 
        refit=True, 
        scoring='recall', 
        n_jobs=-1, 
        
    )
    grid.fit(X_train, y_train)
    print("Grid searching is done!")
    print("The best score: ", grid.best_score_)
    print("The best hyperparameters:")
    print(grid.best_params_)
    return grid


best_model = grid_search_adaboost(Xtrain, ytrain)
final_predictions = best_model.predict(Xtest)
accuracy = accuracy_score(ytest, final_predictions)
precision = precision_score(ytest, final_predictions, average='macro')
recall = recall_score(ytest, final_predictions, average='macro')
f1 = f1_score(ytest, final_predictions, average='macro')

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Grid searching is done!
The best score:  0.7042798062152901
The best hyperparameters:
{'base_estimator': DecisionTreeClassifier(max_depth=2), 'learning_rate': 0.8, 'n_estimators': 800}
Accuracy:  0.9688
Precision:  0.9385655562306492
Recall:  0.8558483223311275
F1 Score:  0.8919205183491941


/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/opt/anaconda3/envs/ds420/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was ren